In [1]:
import requests
import json
import numpy as np
from binance.client import Client
import pandas as pd
from sklearn import preprocessing
import talib

In [2]:
base_url = 'https://api.binance.com'

TRADING_PAIR = 'BTCUSDT'

kline_headers = ['open_time','open','high','low','close','volume','close_time','quote_asset_volume','num_trades','buy_volume','quote_volume','ignore']

In [3]:
client = Client()

In [4]:
nparr = np.zeros((144000,12))
for i,kline in enumerate(client.get_historical_klines_generator(TRADING_PAIR, '1m', "10 day ago UTC")):
    nparr[i] = kline

In [6]:
df = pd.DataFrame(nparr, columns = kline_headers)
df = df[df['open_time']!=0]

In [86]:
starttime = int(df.open_time.min()/1000) 
endtime= int(df.close_time.max()/1000)
df.to_csv(f'kline_{starttime}_{endtime}.csv')

In [6]:
df.columns

Index(['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time',
       'quote_asset_volume', 'num_trades', 'buy_volume', 'quote_volume',
       'ignore'],
      dtype='object')

List of TA indicators. Do for all open, high, low, close:
- SMA (different timeframes)
- EMAs (different timeframes)
- RSI
- MACD
- Momentum (different timeframe)

In [7]:
# Scales all columns. Maybe do indexing with the open times.
x = df.values 
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled, columns = kline_headers)

In [13]:
talib_funcs = talib.get_function_groups()
print(talib_funcs)

{'Cycle Indicators': ['HT_DCPERIOD', 'HT_DCPHASE', 'HT_PHASOR', 'HT_SINE', 'HT_TRENDMODE'], 'Math Operators': ['ADD', 'DIV', 'MAX', 'MAXINDEX', 'MIN', 'MININDEX', 'MINMAX', 'MINMAXINDEX', 'MULT', 'SUB', 'SUM'], 'Math Transform': ['ACOS', 'ASIN', 'ATAN', 'CEIL', 'COS', 'COSH', 'EXP', 'FLOOR', 'LN', 'LOG10', 'SIN', 'SINH', 'SQRT', 'TAN', 'TANH'], 'Momentum Indicators': ['ADX', 'ADXR', 'APO', 'AROON', 'AROONOSC', 'BOP', 'CCI', 'CMO', 'DX', 'MACD', 'MACDEXT', 'MACDFIX', 'MFI', 'MINUS_DI', 'MINUS_DM', 'MOM', 'PLUS_DI', 'PLUS_DM', 'PPO', 'ROC', 'ROCP', 'ROCR', 'ROCR100', 'RSI', 'STOCH', 'STOCHF', 'STOCHRSI', 'TRIX', 'ULTOSC', 'WILLR'], 'Overlap Studies': ['BBANDS', 'DEMA', 'EMA', 'HT_TRENDLINE', 'KAMA', 'MA', 'MAMA', 'MAVP', 'MIDPOINT', 'MIDPRICE', 'SAR', 'SAREXT', 'SMA', 'T3', 'TEMA', 'TRIMA', 'WMA'], 'Pattern Recognition': ['CDL2CROWS', 'CDL3BLACKCROWS', 'CDL3INSIDE', 'CDL3LINESTRIKE', 'CDL3OUTSIDE', 'CDL3STARSINSOUTH', 'CDL3WHITESOLDIERS', 'CDLABANDONEDBABY', 'CDLADVANCEBLOCK', 'CDLBELTHO

In [12]:
talib.CDLABANDONEDBABY()

In [9]:
close = df['close'].to_numpy()

In [21]:
talib.SMA(close, 10)

array([      nan,       nan,       nan, ..., 33225.382, 33234.227,
       33239.992])

In [25]:
talib.SMA(df['close'],50).isna().sum()

49